In [1]:
# Snowpark
import snowflake.snowpark
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import sproc, udf, udtf, col, call_table_function
from snowflake.snowpark.session import Session
from snowflake.snowpark import types as T
from snowflake.snowpark.window import Window
import json

import pandas as pd
import datetime
from prophet import Prophet
from prophet.plot import plot_plotly

with open('creds.json') as f:
    connection_parameters = json.load(f)


In [2]:

session = Session.builder.configs(connection_parameters).create()
session.add_packages('snowflake-snowpark-python', 'scikit-learn', 'pandas', 'numpy', 'joblib', 'cachetools')

The version of package cachetools in the local environment is 5.2.0, which does not fit the criteria for the requirement cachetools. Your UDF might not work when the package version is different between the server and your local environment


In [3]:
session.sql("CREATE OR REPLACE STAGE MODEL").collect()

[Row(status='Stage area MODEL successfully created.')]

### Explore Corn Price history

In [4]:
corn_df = session.table('corn_price_daily')

In [5]:
corn_df.count()

11128

In [6]:
corn_df.describe().to_pandas()

,SUMMARY,COMMODITY,COMMODITY_NAME,SYMBOL,VALUE,UNITS
0,max,KN.AGR42,US Corn Futures,ZC,831.250000,USD per 1 Bushel
1,stddev,None,None,None,146.407447,None
2,min,KN.AGR42,US Corn Futures,ZC,142.750000,USD per 1 Bushel
3,count,11128,11128,11128,11128.000000,11128
4,mean,None,None,None,340.053779,None


In [7]:
corn_df.limit(100).to_pandas()

,COMMODITY,COMMODITY_NAME,SYMBOL,DATE,VALUE,UNITS
0,KN.AGR42,US Corn Futures,ZC,1979-12-27,289.25,USD per 1 Bushel
1,KN.AGR42,US Corn Futures,ZC,1979-12-28,291.25,USD per 1 Bushel
2,KN.AGR42,US Corn Futures,ZC,1979-12-31,289.20,USD per 1 Bushel
3,KN.AGR42,US Corn Futures,ZC,1980-01-02,286.50,USD per 1 Bushel
4,KN.AGR42,US Corn Futures,ZC,1980-01-03,286.50,USD per 1 Bushel
...,...,...,...,...,...,...
95,KN.AGR42,US Corn Futures,ZC,1980-05-15,273.25,USD per 1 Bushel
96,KN.AGR42,US Corn Futures,ZC,1980-05-16,271.25,USD per 1 Bushel
97,KN.AGR42,US Corn Futures,ZC,1980-05-19,272.00,USD per 1 Bushel
98,KN.AGR42,US Corn Futures,ZC,1980-05-20,271.75,USD per 1 Bushel


### Train Prophet forecasting model

In [8]:
# get data to pandas
train_df = corn_df.select('DATE','VALUE').to_pandas()
train_df.columns = ['ds', 'y']

In [9]:
m = Prophet()
m.fit(train_df)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/jgriffith/opt/anaconda3/envs/python38/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -266.764
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99         19047     0.0617229       1856.89      0.9647      0.9647      120   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       20358.6      0.032306       1565.39      0.4455           1      232   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       20951.3     0.0259927       2714.52           1           1      337   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       21265.8     0.0472744       1904.33           1           1      447   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       21656.3     0.0046457       468.828           1           1      557   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [10]:
future = m.make_future_dataframe(periods=365)
forecast = m.predict(future)

/Users/jgriffith/opt/anaconda3/envs/python38/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
/Users/jgriffith/opt/anaconda3/envs/python38/lib/python3.8/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


In [11]:
forecast.loc[forecast['ds'] > datetime.datetime(2021, 1, 1, 0, 0, 0)]

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
10575,2021-01-04,506.929927,426.816682,585.418906,506.929927,506.929927,-1.941577,-1.941577,-1.941577,1.087242,1.087242,1.087242,-3.028818,-3.028818,-3.028818,0.0,0.0,0.0,504.988350
10576,2021-01-05,507.019743,429.271835,583.236262,507.019743,507.019743,-1.427423,-1.427423,-1.427423,1.376658,1.376658,1.376658,-2.804082,-2.804082,-2.804082,0.0,0.0,0.0,505.592320
10577,2021-01-06,507.109560,430.062258,578.326613,507.109560,507.109560,-1.123614,-1.123614,-1.123614,1.479052,1.479052,1.479052,-2.602665,-2.602665,-2.602665,0.0,0.0,0.0,505.985946
10578,2021-01-07,507.199376,431.976691,584.510556,507.199376,507.199376,-1.467316,-1.467316,-1.467316,0.956681,0.956681,0.956681,-2.423997,-2.423997,-2.423997,0.0,0.0,0.0,505.732061
10579,2021-01-08,507.289193,430.021630,587.590167,507.289193,507.289193,-0.752807,-0.752807,-0.752807,1.514418,1.514418,1.514418,-2.267225,-2.267225,-2.267225,0.0,0.0,0.0,506.536386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11488,2023-12-09,602.943804,492.977423,656.940957,580.954960,622.776193,-27.758019,-27.758019,-27.758019,-14.139626,-14.139626,-14.139626,-13.618393,-13.618393,-13.618393,0.0,0.0,0.0,575.185785
11489,2023-12-10,603.033621,505.137996,679.520344,580.386720,623.103198,-5.576856,-5.576856,-5.576856,7.725575,7.725575,7.725575,-13.302431,-13.302431,-13.302431,0.0,0.0,0.0,597.456764
11490,2023-12-11,603.123437,507.728778,676.625312,580.224197,623.430202,-11.873573,-11.873573,-11.873573,1.087242,1.087242,1.087242,-12.960814,-12.960814,-12.960814,0.0,0.0,0.0,591.249864
11491,2023-12-12,603.213254,509.971240,674.855215,580.224001,623.757207,-11.219084,-11.219084,-11.219084,1.376658,1.376658,1.376658,-12.595742,-12.595742,-12.595742,0.0,0.0,0.0,591.994170


In [12]:
plot_plotly(m, forecast)

In [14]:
class Forecast: 
      def __init__(self):
            self.n_periods = None
            
      def process(self, n_periods:int):
            future = m.make_future_dataframe(periods=n_periods)
            forecast = m.predict(future)
            
            forecast = forecast[['ds','yhat', 'yhat_upper', 'yhat_lower']]

            for row in forecast.itertuples():
                  yield (row[1].date(), row[2], row[3], row[4])
                  
forecast_udtf = udtf(
      Forecast, 
      name='forecast', 
      input_types=[T.IntegerType()], 
      output_schema=T.StructType([T.StructField('date', T.DateType()), 
                              T.StructField('yhat', T.FloatType()),
                              T.StructField('yhat_upper', T.FloatType()),
                              T.StructField('yhat_lower', T.FloatType())]),
      is_permanent=True,
      packages=['prophet','pandas'],
      stage_location='@model',
      replace=True
)

INFO:snowflake.connector.cursor:query: [ls '@model']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01a8f707-0000-de52-00...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT  *  FROM ( SELECT  *  FROM (information_schema.packages)) WHERE (("LANGUA...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "PACKAGE_NAME", array_agg("VERSION") AS "ARRAY_AGG(VERSION)" FROM ( SELEC...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [PUT 'file:///tmp/placeholder/udf_py_349493816.zip' '@model/forecast' PARALLEL = ...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE FUNCTION forecast(arg1 INT) RETURNS TABLE (date DATE,yhat FLOA...]
INFO:snowflake.connector.cursor:query execution done


In [15]:
session.table_function(forecast_udtf(F.lit(30))).filter((F.col("DATE") > F.current_date())).to_pandas()

INFO:snowflake.connector.cursor:query: [SELECT  *  FROM ( SELECT  *  FROM ( TABLE (forecast(30 :: INT) ))) WHERE ("DATE"...]
INFO:snowflake.connector.cursor:query execution done


,DATE,YHAT,YHAT_UPPER,YHAT_LOWER
0,2022-12-15,560.381053,639.069499,493.105618
1,2022-12-16,561.467158,635.614671,490.526315
2,2022-12-17,546.352333,623.130047,475.307912
3,2022-12-18,568.765087,645.368619,492.723320
4,2022-12-19,562.680113,640.396546,481.820945
5,2022-12-20,563.526165,640.093960,487.508702
6,2022-12-21,564.185949,638.942528,486.774271
7,2022-12-22,564.219231,639.720225,489.838329
8,2022-12-23,565.328424,642.090303,486.153808
9,2022-12-24,550.219239,621.068270,476.201391


### Create Stored Procedure for ongoing trainging of Prophet model

In [16]:
actuals = session.table('corn_price_daily').select(F.col('DATE'),F.col('VALUE')).to_pandas() 
actuals.columns = ['ds', 'actual']

forecast.set_index('ds').join(actuals.set_index('ds'), how='left')


INFO:snowflake.connector.cursor:query: [SELECT "DATE", "VALUE" FROM ( SELECT  *  FROM (corn_price_daily))]
INFO:snowflake.connector.cursor:query execution done


,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,actual
ds,,,,,,,,,,,,,,,,,,,
1979-12-27,309.967589,224.197174,376.282206,309.967589,309.967589,-4.993774,-4.993774,-4.993774,0.956681,0.956681,0.956681,-5.950455,-5.950455,-5.950455,0.0,0.0,0.0,304.973815,289.25
1979-12-28,309.955250,228.545098,380.586617,309.955250,309.955250,-4.025625,-4.025625,-4.025625,1.514418,1.514418,1.514418,-5.540043,-5.540043,-5.540043,0.0,0.0,0.0,305.929625,291.25
1979-12-31,309.918234,228.310903,384.867933,309.918234,309.918234,-3.328009,-3.328009,-3.328009,1.087242,1.087242,1.087242,-4.415250,-4.415250,-4.415250,0.0,0.0,0.0,306.590226,289.20
1980-01-02,309.893557,230.765067,379.546233,309.893557,309.893557,-2.289328,-2.289328,-2.289328,1.479052,1.479052,1.479052,-3.768380,-3.768380,-3.768380,0.0,0.0,0.0,307.604229,286.50
1980-01-03,309.881219,231.252646,385.555685,309.881219,309.881219,-2.522233,-2.522233,-2.522233,0.956681,0.956681,0.956681,-3.478915,-3.478915,-3.478915,0.0,0.0,0.0,307.358985,286.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-09,602.943804,492.977423,656.940957,580.954960,622.776193,-27.758019,-27.758019,-27.758019,-14.139626,-14.139626,-14.139626,-13.618393,-13.618393,-13.618393,0.0,0.0,0.0,575.185785,NaN
2023-12-10,603.033621,505.137996,679.520344,580.386720,623.103198,-5.576856,-5.576856,-5.576856,7.725575,7.725575,7.725575,-13.302431,-13.302431,-13.302431,0.0,0.0,0.0,597.456764,NaN
2023-12-11,603.123437,507.728778,676.625312,580.224197,623.430202,-11.873573,-11.873573,-11.873573,1.087242,1.087242,1.087242,-12.960814,-12.960814,-12.960814,0.0,0.0,0.0,591.249864,NaN


In [28]:
train_df = (session.table('corn_price_daily')
            .with_column('ROWNUM', F.row_number().over(Window.order_by(F.col('DATE').desc()) ) )
            .filter(F.col('ROWNUM') < 10)
            .select('DATE','VALUE')
           ).to_pandas()
train_df

INFO:snowflake.connector.cursor:query: [SELECT  *  FROM (corn_price_daily)]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "DATE", "VALUE" FROM ( SELECT  *  FROM ( SELECT "COMMODITY", "COMMODITY_N...]
INFO:snowflake.connector.cursor:query execution done


,DATE,VALUE
0,2022-12-13,654.88
1,2022-12-12,641.25
2,2022-12-09,634.75
3,2022-12-08,642.50
4,2022-12-07,641.25
5,2022-12-06,637.25
6,2022-12-05,640.50
7,2022-12-02,646.25
8,2022-12-01,650.00


In [29]:
def train_prophet(session: snowflake.snowpark.Session, n_periods:int = 20*365) -> str:
    # gether data
    train_df = (session.table('corn_price_daily')
                .with_column('ROWNUM', F.row_number().over(Window.order_by(F.col('DATE').desc()) ) )
                .filter(F.col('ROWNUM') < n_periods)
                .select('DATE','VALUE')
                ).to_pandas()
    train_df.columns = ['ds','y']

    #fit model
    m = Prophet()
    m.fit(train_df)

    # register newly trained model as udtf for ongoing inference
    class Forecast: 
        def process(self, n_periods:int):
                future = m.make_future_dataframe(periods=n_periods)
                forecast = m.predict(future)
                
                forecast = forecast[['ds','yhat', 'yhat_upper', 'yhat_lower']]

                for row in forecast.itertuples():
                    yield (row[1].date(), row[2], row[3], row[4])
                    
    forecast_udtf = udtf(
        Forecast, 
        name='forecast', 
        input_types=[T.IntegerType()], 
        output_schema=T.StructType([T.StructField('date', T.DateType()), 
                                    T.StructField('yhat', T.FloatType()),
                                    T.StructField('yhat_upper', T.FloatType()),
                                    T.StructField('yhat_lower', T.FloatType())]),
        is_permanent=True,
        packages=['prophet','pandas'],
        stage_location='@model',
        replace=True
    )


    return "Model trained."
# register function as sproc
train_prophet = sproc(train_prophet, 
                      name='train_prophet', 
                      stage_location='@model', 
                      is_permanent=True, 
                      replace=True, 
                      packages=['prophet', 'snowflake-snowpark-python', 'pandas', 'joblib'])

INFO:snowflake.connector.cursor:query: [ls '@model']
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "name" FROM ( SELECT  *  FROM  TABLE ( RESULT_SCAN('01a8f73a-0000-df07-00...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT  *  FROM ( SELECT  *  FROM (information_schema.packages)) WHERE (("LANGUA...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [SELECT "PACKAGE_NAME", array_agg("VERSION") AS "ARRAY_AGG(VERSION)" FROM ( SELEC...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE PROCEDURE train_prophet(arg1 BIGINT) RETURNS STRING LANGUAGE P...]
INFO:snowflake.connector.cursor:query execution done


In [30]:
session.sql("call train_prophet()").collect()

INFO:snowflake.connector.cursor:query: [call train_prophet()]
INFO:snowflake.connector.cursor:query execution done


[Row(TRAIN_PROPHET='Model trained.')]

In [31]:
session.call('train_prophet')

INFO:snowflake.connector.cursor:query: [CALL train_prophet()]
INFO:snowflake.connector.cursor:query execution done


'Model trained.'

In [20]:
session.sql('select * from table(forecast(30))').to_pandas()

INFO:snowflake.connector.cursor:query: [select * from table(forecast(30))]
INFO:snowflake.connector.cursor:query execution done


,DATE,YHAT,YHAT_UPPER,YHAT_LOWER
0,1979-12-27,305.891001,381.071846,234.094721
1,1979-12-28,306.842170,381.957354,234.748791
2,1979-12-31,307.494891,384.306692,229.310334
3,1980-01-02,308.504203,381.972402,238.992103
4,1980-01-03,308.254807,382.205378,229.386806
...,...,...,...,...
11153,2023-01-08,578.487697,657.518527,502.029012
11154,2023-01-09,572.074245,646.174120,496.331343
11155,2023-01-10,572.583015,653.776019,493.617249
11156,2023-01-11,572.880606,644.621314,498.764176
